# Profitable App Profiles For The App Store And Google Play Markets

Project goal is to analyze data to help understand what kinds of app are likely to attract more users

In [ ]:
opened_apple_file = open('AppleStore.csv')
opened_google_file = open('googleplaystore.csv')
from csv import reader

read_apple_file = reader(opened_apple_file)
read_google_file = reader(opened_google_file)
apps_apple_data = list(read_apple_file)
apps_google_data = list(read_google_file)

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) after each row
        
        if rows_and_columns:
            print('Number of rows:', len(dataset))
            print('Number of columns', len(dataset[0]))
            
explore_data(apps_apple_data, 1, 2)
explore_data(apps_google_data, 1, 2)
print(len(apps_apple_data))
print(len(apps_apple_data[0]))


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


7198
17


# Data cleaning

Error in row 10473. It shows an app rating of 19 as opposed to a maximum of 5.


In [2]:
print(apps_google_data[10473]) # row with error
print('\n')
print(apps_google_data[0]) # Data Header
print('\n')
print(apps_google_data[1]) # row with correct data

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


In [3]:
print(len(apps_google_data[1:])) # number of apps in data set
del apps_google_data[10473]
print(len(apps_google_data[1:])) # number of apps in after wrong data is deleted

10841
10840


# Find duplicate entries

Google play data has duplicate entries

In [4]:
duplicate_apps = []
unique_apps = []

for app in apps_google_data[1:]:
    app_name = app[0]
    if app_name in unique_apps:
        duplicate_apps.append(app_name)
    else:
        unique_apps.append(app_name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Example of duplicate apps', duplicate_apps[1:5])

Number of duplicate apps: 1181


Example of duplicate apps ['Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


# Removing duplicate entries

Number of duplicate entires is 1180. 
Expected number of entries without duplicates 10839 - 1180 = 9659.

- Made an empty dictionary which I will use to store unique apps and with the largest user reviews count


In [5]:
reviews_max = {}

for app in apps_google_data[1:]:
    app_name = app[0]
    n_reviews = float(app[3])
    if app_name in reviews_max and reviews_max[app_name] < n_reviews:
        reviews_max[app_name] = n_reviews
        
    elif app_name not in reviews_max:
        reviews_max[app_name] = n_reviews
        
print(len(reviews_max))


9659


Use reviews_max dictionary to remove duplicates.

In [6]:
apps_google_sanitized = []
apps_google_added = []

for app in apps_google_data[1:]:
    app_name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[app_name] == n_reviews) and (app_name not in apps_google_added):
        apps_google_sanitized.append(app)
        apps_google_added.append(app_name)
        
print(len(apps_google_sanitized))

9659


# Identify means to detect non-english apps

loop through each character and use pythons built in ord() function to check if the character is a letter of the alphabet.

- Watch out for emojis and symbols

In [7]:
def is_english(a_string):
    
    for character in a_string:
        if ord(character) > 127:
            return False
    return True

print(is_english('Instagram'))
print(is_english('《欢乐颂2》电视剧热播'))

True
False


# Filter out non-english apps

- remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range which in this case is anything over 127 ASCII code. 
- This means all English apps with up to three emoji or other special characters will still be labeled as English.


In [8]:
def is_english(a_string):
    non_ascii = 0
    
    for character in a_string:
        if ord(character) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else:
        return True
    
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


With proof of concept working we can now proceed to filter both data sets (apps_google_data and apps_apple_data)

In [9]:
google_apps_english = []
apple_apps_english = []

for app in apps_google_sanitized:
    app_name = app[0]
    if is_english(app_name):
        google_apps_english.append(app)
        
for app in apps_apple_data[1:]:
    app_name = app[1]
    if is_english(app_name):
        apple_apps_english.append(app)
        
explore_data(google_apps_english, 0, 4)
print('\n')
print('\n')
print('\n')
explore_data(apple_apps_english, 0, 4)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']








['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2'

# Filter out free apps from both data sets

- We are constrained to building free apps thus we have to loop through both data sets from the android and ios stores to determine free apps.

In [10]:
apps_google_final = []
apps_apple_final = []

for app in google_apps_english:
    price = app[7]
    if price == '0':
        apps_google_final.append(app)
        
for app in apple_apps_english:
    price = app[5]
    if price == '0':
        apps_apple_final.append(app)
        
print(len(apps_google_final))
print(len(apps_apple_final))    

8864
4056


# Analyze the frequency tables

- One function to generate frequency tables that show percentages
- Another function we can use to display the percentages in a descending order

In [11]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage
        
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

# Find most common genre 
- Apple store

In [12]:
display_table(apps_apple_final, -5)

Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


- Google play store

In [13]:
display_table(apps_google_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

# Most Popular Apps in the App store

In [14]:
apple_store_genres = freq_table(apps_apple_final, -5)

for genre in apple_store_genres:
    total = 0
    num_of_apps_in_genre = 0
    for app in apps_apple_final:
        app_genre = app[-5]
        if app_genre == genre:
            num_of_ratings = float(app[6])
            total += num_of_ratings
            num_of_apps_in_genre += 1
    avg_ratings = total / num_of_apps_in_genre
    print(genre, ':', avg_ratings)

    

Productivity : 19053.887096774193
Weather : 47220.93548387097
Shopping : 18746.677685950413
Reference : 67447.9
Finance : 13522.261904761905
Music : 56482.02985074627
Utilities : 14010.100917431193
Travel : 20216.01785714286
Social Networking : 53078.195804195806
Sports : 20128.974683544304
Health & Fitness : 19952.315789473683
Games : 18924.68896765618
Food & Drink : 20179.093023255813
News : 15892.724137931034
Book : 8498.333333333334
Photo & Video : 27249.892215568863
Entertainment : 10822.961077844311
Business : 6367.8
Lifestyle : 8978.308510638299
Education : 6266.333333333333
Navigation : 25972.05
Medical : 459.75
Catalogs : 1779.5555555555557


# Most Popular apps in the play store

In [15]:
categories_android = freq_table(apps_google_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in apps_google_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_